# Working Notebook

__Phase 3 Project - Chicago Traffic Crash Classification__

__Business Understanding__

__Data Understanding and Preparation__

Load dependencies

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Import data files

In [15]:
crashes = pd.read_csv('Chicago-Crashes/data/Traffic_Crashes_-_Crashes.csv')
people = pd.read_csv('Chicago-Crashes/data/Traffic_Crashes_-_People.csv')
vehicles = pd.read_csv('Chicago-Crashes/data/Traffic_Crashes_-_Vehicles.csv')

In [11]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   RD_NO                          536550 non-null  object 
 2   CRASH_DATE_EST_I               41029 non-null   object 
 3   CRASH_DATE                     541142 non-null  object 
 4   POSTED_SPEED_LIMIT             541142 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 6   DEVICE_CONDITION               541142 non-null  object 
 7   WEATHER_CONDITION              541142 non-null  object 
 8   LIGHTING_CONDITION             541142 non-null  object 
 9   FIRST_CRASH_TYPE               541142 non-null  object 
 10  TRAFFICWAY_TYPE                541142 non-null  object 
 11  LANE_CNT                       198968 non-null  float64
 12  ALIGNMENT                     

In [16]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195747 entries, 0 to 1195746
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1195747 non-null  object 
 1   PERSON_TYPE            1195747 non-null  object 
 2   CRASH_RECORD_ID        1195747 non-null  object 
 3   RD_NO                  1185613 non-null  object 
 4   VEHICLE_ID             1172146 non-null  float64
 5   CRASH_DATE             1195747 non-null  object 
 6   SEAT_NO                244512 non-null   float64
 7   CITY                   879728 non-null   object 
 8   STATE                  890189 non-null   object 
 9   ZIPCODE                803192 non-null   object 
 10  SEX                    1177665 non-null  object 
 11  AGE                    852450 non-null   float64
 12  DRIVERS_LICENSE_STATE  705542 non-null   object 
 13  DRIVERS_LICENSE_CLASS  608027 non-null   object 
 14  SAFETY_EQUIPMENT  

In [18]:
percent_missing = people.isnull().sum() * 100 / len(people)
missing_value_df = pd.DataFrame({'column_name': people.columns,
                                 'percent_missing': percent_missing})

pd.set_option('display.max_rows', None)
missing_value_df

,column_name,percent_missing
PERSON_ID,PERSON_ID,0.000000
PERSON_TYPE,PERSON_TYPE,0.000000
CRASH_RECORD_ID,CRASH_RECORD_ID,0.000000
RD_NO,RD_NO,0.847504
VEHICLE_ID,VEHICLE_ID,1.973745
CRASH_DATE,CRASH_DATE,0.000000
SEAT_NO,SEAT_NO,79.551527
CITY,CITY,26.428584
STATE,STATE,25.553733
ZIPCODE,ZIPCODE,32.829269
